# Introduction

This Project is based on a problem about a Restaurant Market place called Zomato, this company is an intermediate between customers and restaurants.

As a study case, we were hired to help the new CEO to understand how Zomato numbers are going, He sent to us a list of question that must be answered as soon as possible in an online dashboard to show a visual knowledge from the data for a better understanding.


## Imports

In [1]:
from IPython.core.display   import HTML
from forex_python.converter import CurrencyRates
from datetime               import datetime
from folium.plugins         import MarkerCluster, ScrollZoomToggler
from folium.map             import LayerControl

import numpy                as np
import pandas               as pd
import seaborn              as sns
import matplotlib.pyplot    as plt
import plotly.express       as px
import plotly.graph_objects as go
import inflection
import folium


## Useful Functions

In [2]:
# This Function create a better visual set to Jupyter

def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

jupyter_settings()

# Fulfill the the country names with the matching ID

countries = {
    1:'India',
    14:'Australia',
    30:'Brazil',
    37:'Canada',
    94:'Indonesia',
    148:'New Zeland',
    162:'Philippines',
    166:'Qatar',
    184:'Singapure',
    189:'South Africa',
    191:'Sri Lanka',
    208:'Turkey',
    214:'United Arab Emirates',
    215:'England',
    216:'United States of America'
}

def country_name( country_id ):
    return countries[country_id]

# Create food type

def price_type( price_range ):
    if price_range == 1:
        return 'cheap'
    elif price_range == 2:
        return 'normal'
    elif price_range == 3:
        return 'expensive'
    else:
        return 'gourmet'

# Colors

colors = {
    '3F7E00': 'darkgreen',
    '5BA829': 'green',
    '9ACD32': 'lightgreen',
    'CDD614': 'orange',
    'FFBA00': 'red',
    'CBCBC8': 'darkred',
    'FF7800': 'darkred'
}

def color_name( color_code ):
    return colors[color_code]

def rename_columns( dataframe ):
    df = dataframe.copy()
    title = lambda x: inflection.titleize( x )
    snakecase = lambda x: inflection.underscore( x )
    spaces = lambda x: x.replace( ' ', '' )
    cols_old = list( df.columns )
    cols_old = list( map( title, cols_old ) )
    cols_old = list( map( spaces, cols_old ) )
    cols_new = list( map( snakecase, cols_old ) )
    df.columns = cols_new
    
    return df

# Your supervisor asked you to initially to consider all the restaurants only by one kind of cuisine

# df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

# I've decide to convert the values to USD dollar, and add a new column with the value converted

currencies = {
    'Philippines':'PHP',
    'Brazil':'BRL',
    'Australia':'AUD',
    'United States of America':'USD',
    'Canada':'CAD',
    'Singapure':'SGD',
    'United Arab Emirates':'AED',
    'India':'INR',
    'Indonesia':'IDR',
    'New Zeland':'NZD',
    'England':'GBP',
    'Qatar':'QAR',
    'South Africa':'ZAR',
    'Sri Lanka':'LKR',
    'Turkey':'TRY'
}

def define_currency( currency ):    
    return currencies[currency]

# Create a CurrencyRates object

c = CurrencyRates()

# Example: Convert 100 USD to EUR

usd_amount = 1000
eur_amount = c.convert('EUR', 'USD', usd_amount)
print(f"{usd_amount} USD is equal to {eur_amount} EUR")


%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\gabre\anaconda3\envs\exercises_1\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


ConnectionError: HTTPSConnectionPool(host='theforexapi.com', port=443): Max retries exceeded with url: /api/latest?base=EUR&symbols=USD&rtype=fpy (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001ACE2966040>: Failed to establish a new connection: [WinError 10060] Uma tentativa de conexão falhou porque o componente conectado não respondeu\r\ncorretamente após um período de tempo ou a conexão estabelecida falhou\r\nporque o host conectado não respondeu'))

# Checking Data

In [ ]:
df = pd.read_csv( 'C:\\Users\\gabre\\DS IN PROGRESS\\DS_2023\\Ciclo_Basico\\FTC Analisando dados com Python\\FTC_student_project\\dataset\\zomato.csv' )
df.head()


In [ ]:
# Checking the data type

df.dtypes


In [ ]:
# Look for NaNs

df.isna().sum()


In [ ]:
# Check the datset size, and check if we have duplicated restaurants

print( df.shape ) 
print( df['Restaurant ID'].nunique() )


# Cleaning and modeling the data

In [ ]:
# Using the functions to prepare the columns below:

# We could notice that there are duplicated Restaurants ID

df = df.drop_duplicates()

# Match the countries' name with their code

df['Country Code'] = df['Country Code'].apply( country_name )

# Standardize the currencies

df['Currency'] = df['Country Code'].apply( define_currency )

# Create a column with USD value converted

df['Dollar USD Value'] = df.apply( lambda x: c.convert( x['Currency'], 'USD', x['Average Cost for two'] ), axis=1 )
df['Dollar USD Value'] = df['Dollar USD Value'].round( 2 )

# Create the price range

df['Price range'] = df['Price range'].apply( price_type )

# Match the colors with thir codes

df['Rating color'] = df['Rating color'].apply( color_name )

# Select just the first cuisine tha shows up in the line

df['Cuisines'] = df.loc[:, 'Cuisines'].apply( lambda x: str(x).split( ',' )[0] )

# Rename the columns

df = rename_columns( df )
df.head()


In [ ]:
print( df.shape )
print( df.isna().sum() )
print( df.dtypes )


## Descriptive Statistics

In [ ]:
# We are going to divide the columns into two datasets with different kind of variables ( Numeral and categorical )

num_attributes = df.select_dtypes( include=['float64', 'int64'] )
cat_attributes = df.select_dtypes( exclude=['float64', 'int64', 'datetime64[ns]'] )


### Numerical Attributes

In [ ]:
# Central tendency - mean and median

ct1 = pd.DataFrame( num_attributes.apply( np.mean ) ).T
ct2 = pd.DataFrame( num_attributes.apply( np.median ) ).T

# Dispersion - std, min, max, range, skew, kurtosis

d1 = pd.DataFrame( num_attributes.apply( np.std ) ).T
d2 = pd.DataFrame( num_attributes.apply( min ) ).T
d3 = pd.DataFrame( num_attributes.apply( max ) ).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis() ) ).T

# Concatenate

m = pd.concat( [d2, d3, d4, ct1, ct2, d1, d5, d6] ).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m_rounded = m.round(2)
m_rounded


In [ ]:
# Extract unique values from the 'dollar_usd_value' column

range_values = df['dollar_usd_value'].unique()
print( range_values )

# Sort the list in descending order

range_values_sorted = sorted( range_values, reverse=erse=True )

range_values_sorted


The list before showed that we have only one outlier with the value, which is already converted to USD dollar, 15932993.75, 
so I decided to breake this line value down to 1593.3, it is probably a typing error, but in any case it will be demonstrade to the CEO, after we got a return from
the restaurant if the price is really that

In [ ]:
df.loc[df['dollar_usd_value'] == 15932993.75, 'dollar_usd_value'] = 1593.3
df.loc[df['dollar_usd_value'] == 1593.3]


### Categorical Attributes

In [ ]:
# Check the range of the variables

cat_attributes.apply( lambda x: x.unique().shape[0] )


# CEO's Questions

In [ ]:
# Save a copy of df as CSV and start work with a checkpoint df1

# Get today's date

today_date = datetime.now().strftime("%d-%m-%Y")

# Define the filename

filename = f"zomato_treated_{today_date}.csv"

# Define the directory path

directory_path = r"C:\\Users\\gabre\\DS IN PROGRESS\\DS_2023\\Ciclo_Basico\\FTC Analisando dados com Python\\FTC_student_project\\dataset"

# Construct the full file path

file_path = f"{directory_path}\\{filename}"

# Save DataFrame to CSV

df.to_csv( file_path, index=False )

print(f"DataFrame saved to: {file_path}")

# Checkpoint

df1 = df.copy()


## General View

In [ ]:
# How many unique Restaurants are registered?

print( f'The number of unique restaurants is {df1["restaurant_id"].nunique()}.' )

# How many unique Countries are registered?

print( f'The number of unique Countries is {df1["country_code"].nunique()}.' )

# How many unique Cities are registered?

print( f'The number of unique Cities is {df1["city"].nunique()}.' )
      
# What is the total of reviews?

print( f'The total of reviews is {df1["votes"].sum()}.' )

# What is the total of Cuisines?

print( f'The total of cuisines is {df1["cuisines"].nunique()}.' )
      

## Countries' View

In [ ]:
# Which Country has more registered cities?

q1 =  df1[['country_code', 'city']].groupby( 'country_code' ).nunique().sort_values( 'city', ascending=False ).reset_index()
q1


In [ ]:
# Which Country has more restaurants?

q2 =  df1[['country_code', 'restaurant_id']].groupby( 'country_code' ).count().sort_values( 'restaurant_id', ascending=False ).reset_index()
q2


In [ ]:
# Which is the country with more restaurants where price is equal to 4?

q3 = ( df1.loc[df1['price_range'] == 'gourmet', :][['country_code','restaurant_id']]
                                                  .groupby( 'country_code' ).count().sort_values( 'restaurant_id', ascending=False ).reset_index() )
q3


In [ ]:
# Which country has more kind of cuisines?

q4 = df1[['country_code', 'cuisines']].groupby( 'country_code' ).nunique().sort_values( 'cuisines', ascending=False ).reset_index()
q4


In [ ]:
# Which country has more reviews?

q5 = df1[['country_code', 'votes']].groupby( 'country_code' ).sum().sort_values( 'votes', ascending=False ).reset_index()
q5


In [ ]:
# Which country has more delivery food service?

q6 = ( df1.loc[df1['is_delivering_now'] == 1, :][['country_code', 'is_delivering_now']]
                                                .groupby( 'country_code' ).count().sort_values( 'is_delivering_now', ascending=False ).reset_index() )
q6


In [ ]:
# Which country has more quantity of table booking?

q7 = ( df1.loc[df1['has_table_booking'] == 1, :][['country_code', 'has_table_booking']]
                                                .groupby( 'country_code' ).count().sort_values( 'has_table_booking', ascending=False ).reset_index() )
q7


In [ ]:
# Which country has more review in average?

q8 = df1[['country_code', 'votes']].groupby( 'country_code' ).mean().sort_values( 'votes', ascending=False ).reset_index()
q8.round( 2 )


In [ ]:
# Which country has the highest average review?

q9 = df1[['country_code', 'aggregate_rating']].groupby( 'country_code' ).mean().sort_values( 'aggregate_rating', ascending=False ).reset_index()
q9.round( 2 )


In [ ]:
# Which country has the lowest average review?

q10 = df1[['country_code', 'aggregate_rating']].groupby( 'country_code' ).mean().sort_values( 'aggregate_rating', ascending=True ).reset_index()
q10.round( 2 )


In [ ]:
# Which is the average price for two dishes?

q11 = df1[['country_code', 'average_cost_for_two']].groupby( 'country_code' ).mean().sort_values( 'average_cost_for_two', ascending=True ).reset_index()
q11.round( 2 )

# OBs: The cost is in their local currency, to make a real comparison it must be onverted to an unique one, like dollar for example


In [ ]:
# Now considering USD dollar as pattern

q11 = df1[['country_code', 'dollar_usd_value']].groupby( 'country_code' ).mean().sort_values( 'dollar_usd_value', ascending=True ).reset_index()
q11.round( 2 )


## Cities' View

In [ ]:
# Which city has more restaurants?

q12 = df1[['city', 'restaurant_id']].groupby( 'city' ).count().sort_values( 'restaurant_id', ascending=False ).reset_index()
q12.head( 20 )


In [ ]:
# Which city has more restaurant with an average review higher than 4?

q13 = ( df1.loc[df1['aggregate_rating'] > 4, :][['city', 'restaurant_id', 'aggregate_rating']].groupby( 'city' )
       .agg( {'restaurant_id': 'count', 'aggregate_rating': 'mean'} ).sort_values( 'restaurant_id', ascending=False ).reset_index() )
q13.columns = ['city', 'restaurant_id', 'aggregate_mean_rating']
q13.round( 2 ).head( 20 )


In [ ]:
q13 = df1.loc[df1['aggregate_rating'] > 4, :][['city', 'country_code', 'restaurant_id']]
q13 = q13.groupby( ['city', 'country_code'] ).count().reset_index()
q13

In [ ]:
# Which city has more restaurants with an average review lower than 2.5?

q14 = ( df1.loc[ ( df1['aggregate_rating'] < 2.5) & ( df1['votes'] >= 1 ), :][['city', 'restaurant_id', 'aggregate_rating', 'votes']].groupby( 'city' )
       .agg( {'restaurant_id':'count', 'aggregate_rating':'mean', 'votes':'first'} ).sort_values( 'restaurant_id', ascending=False ).reset_index() )
q14.columns = ['city', 'restaurant_id', 'aggregate_mean_rating', 'votes']
q14.head( 10 ).round( 2 )


In [ ]:
# Which city has the most expensive average dish for two?

q15 = df1[['city', 'average_cost_for_two']].groupby( 'city' ).mean().sort_values( 'average_cost_for_two', ascending=False ).reset_index()
q15.round( 2 )

# again, when it comes to price, we'll need to convert to a pattern currency to be able to fairly analyze it


In [ ]:
# Consider dollar value

q15 = df1[['city', 'dollar_usd_value']].groupby( 'city' ).mean().sort_values( 'dollar_usd_value', ascending=False ).reset_index()
q15.round( 2 )


In [ ]:
# Which city has more types of cuisines?

q16 = df1[['city', 'cuisines']].groupby( 'city' ).nunique().sort_values( 'cuisines', ascending=False ).reset_index()
q16.head( 10 )


In [ ]:
# Which city has more booking table service?

q17 = df1.loc[df1['has_table_booking'] == 1, :][['city', 'restaurant_id']].groupby( 'city' ).count().sort_values( 'restaurant_id', ascending=False ).reset_index()
q17.head( 10 )


In [ ]:
# Which city has more delivery service?

q18 = df1.loc[df1['is_delivering_now'] == 1, :][['city', 'restaurant_id']].groupby( 'city' ).count().sort_values( 'restaurant_id', ascending=False ).reset_index()
q18.head( 10 )


In [ ]:
# Which city has more online ordering service?

q19 = df1.loc[df1['has_online_delivery'] == 1, :][['city', 'restaurant_id']].groupby( 'city' ).count().sort_values( 'restaurant_id', ascending=False ).reset_index()
q19.head( 10 )


## Restaurants' View

In [ ]:
# Which Restaurant has more reviews?

q20 = df1[['restaurant_id', 'restaurant_name', 'votes']].groupby( ['restaurant_id'] ).sum().sort_values( 'votes', ascending=False ).reset_index()
q20.head( 10 )


In [ ]:
# Which restaurant has the highest average review?

q21 = df1[['restaurant_name', 'restaurant_id', 'aggregate_rating']].groupby( 'restaurant_name' ).agg( {'restaurant_id': 'first', 'aggregate_rating': 'mean'} )
q21 = q21.sort_values(by=['aggregate_rating', 'restaurant_id'], ascending=[False, True]).reset_index()
q21.head(10)


In [ ]:
# Which restaurant has the most expensive dish for two?

q22 = df1[['restaurant_name', 'restaurant_id', 'average_cost_for_two']].sort_values( 'average_cost_for_two', ascending=False )
q22.head( 10 )


In [ ]:
# Considering USD dollar

q22 = df1[['restaurant_name', 'restaurant_id', 'dollar_usd_value']].sort_values( 'dollar_usd_value', ascending=False )
q22.head( 10 )


In [ ]:
# Which brazilian cuisine restaurant has the lowest average review?

q23 = ( df1.loc[( df1['cuisines'] == 'Brazilian') & ( df1['votes'] >= 1 ), :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[True, True] ).reset_index() )
q23.head( 20 )


In [ ]:
# Now which brazilian restaurant that has brazilian cuisine is the best reviewed?

q24 = ( df1.loc[( df1['country_code'] == 'Brazil' ) & ( df1['cuisines'] == 'Brazilian' ),:][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[False, True] ).reset_index() )
q24.head( 10 )


In [ ]:
# q25 The restaurants that has online ordering have in average more votes?

print( f"The average votes of restaurants with online service is {df1.loc[df1['has_online_delivery'] == 1, :]['votes'].mean():.2f}." )
print( f" The average votes of restaurants without online service is {df1.loc[df1['has_online_delivery'] == 0, :]['votes'].mean():.2f}." )


In [ ]:
# q26 The restaurants which has booking service, are also the ones with more expensive dishe for two?

print( f"The average price for two dishes in restaurants with booking service is {df1.loc[df1['has_table_booking'] == 1, :]['dollar_usd_value'].mean():.2f}" )
print( f"The average price for two dishes in restaurants without booking service is {df1.loc[df1['has_table_booking'] == 0, :]['dollar_usd_value'].mean():.2f}" )


In [ ]:
# q27 The Japnese cuisene restaurants in USA has a mean price for two dishes higher than the steakhouses?

print( f"The average price for two dishes in Japanese restaurants in USA is {df1.loc[df1['cuisines'] == 'Japanese', :]['dollar_usd_value'].mean():.2f}" )
print( f"The average price for two dishes in BBQ restaurants in USA is {df1.loc[df1['cuisines'] == 'BBQ', :]['dollar_usd_value'].mean():.2f}" )


## Cuisines' View

In [ ]:
# Among the restaurants with italian cuisine, which one has the highest aggregate rating?

q28 = ( df1.loc[df1['cuisines'] == 'Italian', :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[False, True] ).reset_index( drop=True ) )
q28.head( 20 )


In [ ]:
# Among the restaurants with italian cuisine, which one has the lowest aggregate rating?

q29 = ( df1.loc[( df1['cuisines'] == 'Italian' ) & ( df1['votes'] >= 1 ), :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[True, True] ).reset_index( drop=True ) )
q29.head( 20 )


In [ ]:
# Among the restaurants with American cuisine, which one has the highest aggregate rating?

q30 = ( df1.loc[df1['cuisines'] == 'American', :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[False, True] ).reset_index( drop=True ) )
q30.head( 20 )


In [ ]:
# Among the restaurants with American cuisine, which one has the lowest aggregate rating?

q31 = ( df1.loc[( df1['cuisines'] == 'American' ) & ( df1['votes'] >= 1 ), :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[True, True] ).reset_index( drop=True ) )
q31.head( 20 )


In [ ]:
# Among the restaurants with Arab cuisine, which one has the highest aggregate rating?

q32 = ( df1.loc[df1['cuisines'] == 'Arabian', :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[False, True] ).reset_index( drop=True ) )
q32.head( 20 )


In [ ]:
# Among the restaurants with Arab cuisine, which one has the lowest aggregate rating?

q33 = ( df1.loc[( df1['cuisines'] == 'Arabian' ) & ( df1['votes'] >= 1 ), :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[True, True] ).reset_index( drop=True ) )
q33.head( 20 )


In [ ]:
# Among the restaurants with Japanese cuisine, which one has the highest aggregate rating?

q34 = ( df1.loc[df1['cuisines'] == 'Japanese', :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[False, True] ).reset_index( drop=True ) )
q34.head( 20 )


In [ ]:
# Among the restaurants with Japanese cuisine, which one has the lowest aggregate rating?

q35 = ( df1.loc[( df1['cuisines'] == 'Japanese' ) & ( df1['votes'] >= 1 ), :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[True, True] ).reset_index( drop=True ) )
q35.head( 20 )


In [ ]:
# Among the restaurants with Home-made cuisine, which one has the highest aggregate rating?

q36 = ( df1.loc[df1['cuisines'] == 'Home-made', :][['restaurant_id', 'restaurant_name', 'aggregate_rating']]
       .sort_values( by=['aggregate_rating', 'restaurant_id'], ascending=[False, True] ).reset_index( drop=True ) )
q36.head( 20 )


In [ ]:
# Which cuisine has the highest price for a dish for two people?

q37 = df1[['cuisines', 'dollar_usd_value']].groupby( 'cuisines' ).mean().round( 2 ).sort_values( 'dollar_usd_value', ascending=False ).reset_index()
q37.head( 20 )


In [ ]:
# Which cuisine has the highest aggregate rating?

q38 = df1[['cuisines', 'aggregate_rating']].groupby( 'cuisines' ).mean().round( 2 ).sort_values( 'aggregate_rating', ascending=False ).reset_index()
q38.head( 20 )


In [ ]:
# Which cuisine accept more online service?

q39 = ( df1.loc[df1['has_online_delivery'] == 1, :][['cuisines', 'has_online_delivery']]
       .groupby( 'cuisines' ).count().sort_values( 'has_online_delivery', ascending=False ).reset_index() )
q39
